In [6]:
import redcap
import csv
import datetime
import pandas as pd

# REDCap API endpoint and token
data = {
    'token': 'DE12CA04415CF93BCD68321EBF5040FD',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
df= redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])
data = {
    'token': 'B5C8DD01C47A66E81E154A8AC735D281',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
ck_wk=redcap.Project('https://redcap.mrc.gm:8443/redcap/api/',data['token'])


In [8]:
df_consent=df.export_records(forms=['consent'])
df_consent=pd.DataFrame(df_consent)
df_not_completed_consent=df_consent[
    (df_consent['consent_complete']=='0')&
    (df_consent['ck_wkno']!='')
    
]

In [9]:
df_consent=df_consent[
    (df_consent['cconsent']=='1')|
    (df_consent['cconsent']=='0')

]

In [10]:
df_consent=df_consent[['participant_id','ck_wkno']]

In [11]:
df_withdrawal=df.export_records(forms=['withdrawal'])
df_withdrawal=pd.DataFrame(df_withdrawal)
df_withdrawal=df_withdrawal[
    (df_withdrawal['withdrawal_complete']=='2')|
    (df_withdrawal['withdrawal_complete']=='1')
]
df_withdrawal=df_withdrawal[['participant_id','withdrawal_complete']]

In [12]:
df_withdrawal_consent=pd.merge(df_consent,df_withdrawal,on='participant_id',how='inner').drop_duplicates('participant_id')

In [13]:
sensitization=ck_wk.export_records(forms=['enumeration_and_sensitisation'])

In [14]:
sensitization=pd.DataFrame(sensitization)

In [15]:
sensitization=sensitization[
    (sensitization['enumeration_and_sensitisation_complete']=='2')|
    (sensitization['enumeration_and_sensitisation_complete']=='1')
]

In [16]:
sensitization=sensitization[['wk_ckno','compoundno','enu_village', 'enu_comp','enu_name','enu_mname', 'enu_fname', 'sen_icd_given', 'sen_contact1']]

In [17]:
sensitization=sensitization.rename(columns={ 'wk_ckno':'ck_wkno'})

In [18]:
village_mapping = {
'001':	'DUMBUTO',
'002':	'SANKANDI',
'003':	'NIORO JATTABA',
'004':	'JATTABA',
'005':	'JIFFARONG',
'006':	'BAJANA',
'007':	'KULI KUNDA',
'008':	'JAMARU',
'009':	'BRIKAMANDING',
'010':	'KANTONG KUNDA',
'011':	'JALI',
'013':	'MANDUAR',
'014':	'BANG KULING',
'015':	'GISSAY',
'016':	'TANKULAR',
'017':	'JOLI',
'018':	'KUYANG',
'019':	'BANTASU',
'020':	'SANTAMBA',
'021':	'MISSIRA',
'022':	'TABORANGKOTO',
'023':	'BURONG',
'024':	'JULA KUNDA',
'025':	'KARANTABA',
'026':	'MANDINA',
'027':	'JANNEH KUNDA',
'028':	'KEMOTO',
'029':	'KENEBA',
'030':	'BATELLING',
'031':	'SANDENG',
'032':	'WUDEBA',
'034':	'KENOKOTO',
'035':	'MANARI',
'036':	'NINETEEN',
'040':	'WUROKANG',
'041':	'KWINELLA SANSANKONO',
'042':	'KWINELLA NIA KUNDA',
'043':	'TENDABA',
'044':	'BUMARR',
'045':	'BAMBAKO',
'046':	'KUNDONG MARIAYA',
'047':	'NEMA',
'048':	'KUNDANG NUMU KUNDA',
'049':	'KUNDANG FULA KUNDA',
'050':	'NEMA KUTA',
'051':	'JIRROFF',
'052':	'MADINA ANGALLEH',
'053':	'JATTA KUNDA',
'054':	'MANDINA CENTRAL',
'055':	'SARE SARJO',
'056':	'SIBETO',
'057':	'SARE NDALLA',
'058':	'TABANANI',
'060':	'WILLINGARA',
'061':	'SARE MAMUDU',
'070':	'KOLIOR',
'071':	'JOMARR',
'072':	'JASOBO',
'073':	'SARE MUSA',
'074':	'YORRO JULA',
'075':	'BABOU YAA',
'076':	'MASEMBEH',
'077':	'GENIERE',
'078':	'KAIAF',
'079':	'MADINA KAIAF',
'080':	'SARE SAMBA',
'081':	'MADINA CEESAY KUNDA',
'082':	'NJOLFEN',
'083':	'TORANKA BANTANG',
'084':	'SAREH PATEH',
'085':	'MUNKUTALA'
}

# Assuming sen_log_data is your DataFrame
sensitization['enu_village'] = sensitization['enu_village'].map(village_mapping)

In [19]:
consent2=pd.merge(df_not_completed_consent,sensitization,on='ck_wkno', how='inner').drop_duplicates('ck_wkno')
consent2.to_csv('not_consented.csv',index=False)

In [ ]:
mardge=pd.merge(df_consent,indiGoEden,on='ck_wkno', how='inner')
mardge.to_csv('indiGoEden.csv',index=False)

In [ ]:
#sensitization=sensitization[
#    (sensitization['ck_wkno']!='')
#]
#sensitization=sensitization[['ck_wkno','sen_outcome']]

In [ ]:
#yet_to_be_consented=pd.merge(sensitization,df_not_completed_consent,on='ck_wkno',how='inner')
#yet_to_be_consented.to_csv('yet_to_be_consented.csv',index=False)

In [ ]:
Approaches_sensitization=pd.merge(sensitization,df_DSS,on='ck_wkno',how='inner')
Approaches_sensitization.to_csv('Enumerated_from_DSS_list.csv',index=False)

In [ ]:
unsensitized_sensitization=Approaches_sensitization[
    (Approaches_sensitization['sen_outcome']=='')
]
unsensitized_sensitization.to_csv('un_sensitize_from_DSS_list.csv',index=False)

In [ ]:
sensitization_completed=Approaches_sensitization[
(Approaches_sensitization['sen_outcome']!='')
]
sensitization_completed.to_csv('sensitize_from_DSS_list.csv',index=False)

In [ ]:
sentize_consent=pd.merge(sensitization_completed,completed_consent,on='ck_wkno',how='inner')
sentize_consent.to_csv('sensitize_consented_from_DSS_list.csv',index=False)

In [20]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(consent2):
    # Drop column: 'consent_sign_q9'
    consent2 = consent2.drop(columns=['consent_sign_q9'])
    # Drop column: 'cstaffid'
    consent2 = consent2.drop(columns=['cstaffid'])
    # Drop column: 'cstudy_explained'
    consent2 = consent2.drop(columns=['cstudy_explained'])
    # Drop column: 'cconsent'
    consent2 = consent2.drop(columns=['cconsent'])
    # Drop column: 'cconsentnoreason'
    consent2 = consent2.drop(columns=['cconsentnoreason'])
    # Drop column: 'cotherdetail'
    consent2 = consent2.drop(columns=['cotherdetail'])
    # Drop column: 'cparticipant_signed'
    consent2 = consent2.drop(columns=['cparticipant_signed'])
    # Drop column: 'consent_completedby_q8'
    consent2 = consent2.drop(columns=['consent_completedby_q8'])
    # Drop column: 'consent_form_date_q10'
    consent2 = consent2.drop(columns=['consent_form_date_q10'])
    # Drop column: 'consent_reviewer_q11'
    consent2 = consent2.drop(columns=['consent_reviewer_q11'])
    # Drop column: 'consent_reviewer_sign_q12'
    consent2 = consent2.drop(columns=['consent_reviewer_sign_q12'])
    # Drop column: 'consent_complete'
    consent2 = consent2.drop(columns=['consent_complete'])
    # Filter rows based on column: 'redcap_repeat_instance'
    consent2 = consent2[consent2['redcap_repeat_instance'].apply(str) == "1"]
    # Drop column: 'redcap_repeat_instance'
    consent2 = consent2.drop(columns=['redcap_repeat_instance'])
    # Rename column 'enu_name' to 'participant_name'
    consent2 = consent2.rename(columns={'enu_name': 'participant_name'})
    # Rename column 'enu_mname' to 'mother\' name '
    consent2 = consent2.rename(columns={'enu_mname': 'mother\' name '})
    # Rename column 'enu_fname' to 'Father\'s name '
    consent2 = consent2.rename(columns={'enu_fname': 'Father\'s name '})
    # Drop column: 'sen_icd_given'
    consent2 = consent2.drop(columns=['sen_icd_given'])
    # Rename column 'sen_contact1' to 'contact'
    consent2 = consent2.rename(columns={'sen_contact1': 'contact'})
    # Replace missing values with "" in columns: 'participant_id', 'redcap_event_name' and 9 other columns
    consent2 = consent2.fillna({'participant_id': "", 'redcap_event_name': "", 'redcap_repeat_instrument': "", 'ck_wkno': "", 'compoundno': "", 'enu_village': "", 'enu_comp': "", 'participant_name': "", "mother' name ": "", "Father's name ": "", 'contact': ""})
    return consent2

consent2_clean = clean_data(consent2.copy())
consent2_clean.head()

,participant_id,redcap_event_name,redcap_repeat_instrument,ck_wkno,compoundno,enu_village,enu_comp,participant_name,mother' name,Father's name,contact
0,EDN1220-T,recruitment_arm_1,consent,02-048-004K,02-048,SANKANDI,048,Amadou Jawo,Sira Jallow,Defala Jawo,
1,EDN1530-J,recruitment_arm_1,consent,25-020-026S,25-020,KARANTABA,020,Natoma Bajinka,Jula Kambi,Karamo Bajinka,3257941
3,EDN1742-H,recruitment_arm_1,consent,52-009-02,52-009,MADINA ANGALLEH,009,HAJJA KOLLEY,ADAM JOBE,EBRIMA KOLLEY,7456977
4,EDN1743-E,recruitment_arm_1,consent,52-013-03,52-013,MADINA ANGALLEH,013,kaddijatou jallow,isatou sey,alkali jallow,2153675
5,EDN1744-C,recruitment_arm_1,consent,52-015-01,52-015,MADINA ANGALLEH,015,mariama bah,hawa bah,Sheriff bah,2374588


In [ ]:
bjdkdkjdldj